# 기상청 대전 데이터셋 병합

- 파일이 연도별, 데이터 종류별, 읍면동별로 나누어져 있음
- 파일 내부에는 각 월 별로 데이터가 한줄 띄어서 구분되어 있음
- 절대 최종 파일인 Daejeon.csv를 Git에 **업로드(Stage & Push)하지 말 것**

In [1]:
import pandas as pd

# 읍면동에 대한 데이터 불러오기
df = pd.read_csv('dataset/Daejeon/대전 읍면동.csv', encoding='cp949')
emds = list(df['읍면동'])

# 데이터 종류에 대한 데이터
types = ['강수','기온','뇌전','습도','풍속','풍향','하늘상태']

# 데이터 연도에 대한 데이터
years = ['2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022']

dfs = []
# 읍면동을 차례로 불러오기
for emd in emds:
    dfs2 = []

    # 연도 데이터를 차례로 불러오기
    for year in years:

        # 데이터 종류를 차례로 불러오기
        for datatype in types: 
            
            # 2010년은 6월부터, 2022년은 4월까지만의 데이터가 존재하므로 아래와 같이 파일명 구분
            if year == '2010':
                df_temp = pd.read_csv('dataset/Daejeon/'+emd+'_'+datatype+'_'+year+'06_'+year+'12.csv')
            elif year == '2022':
                df_temp = pd.read_csv('dataset/Daejeon/'+emd+'_'+datatype+'_'+year+'01_'+year+'04.csv')
            else:
                df_temp = pd.read_csv('dataset/Daejeon/'+emd+'_'+datatype+'_'+year+'01_'+year+'12.csv')
            
            # 컬럼명을 통일시켜줌
            df_temp.columns = ['day','hour',datatype]

            # 강수량에 대한 데이터프레임을 기준으로 나머지 데이터의 컬럼을 추가
            if datatype == '강수':
                df = df_temp
            else:
                df[datatype] = df_temp[datatype]

        # 원본 데이터에서 월을 구분하는 부분은 첫행을 제외하고 나머지 부분에 None이 저장되어있음
        # 해당하는 행 번호를 추출하여 월 구분 및 이후 해당 행 제거 할 때 사용
        del_row = list(df[df.iloc[:,2].isna()].index)

        # 데이터에 몇 월의 데이터인지를 추가
        # 이때 2010년의 경우 6월부터 데이터가 존재하므로 아래와 같이 구분
        if year == '2010':
            j=0
            for n,i in enumerate(del_row + [len(df)-1]):
                df.loc[j:i,'month'] = n+6
                j=i+1
        else:
            j=0
            for n,i in enumerate(del_row + [len(df)-1]):
                df.loc[j:i,'month'] = n+1
                j=i+1

        # 월이 적혀있는 필요없는 행 제거
        df.drop(del_row, axis=0, inplace=True)
        df['year'] = int(year)

        # 연도별로 구분된 데이터를 dfs2에 추가
        dfs2.append(df)
    
    # dfs2에 저장된 연도별 데이터를 병합
    df2 = pd.concat(dfs2, ignore_index=True)

    # 읍면동에 해당하는 컬럼을 추가
    df2['읍면동'] = emd

    # 읍면동별로 구분된 데이터를 dfs에 추가 
    dfs.append(df2)

# dfs에 존재하는 데이터를 최종적으로 병합
df_final = pd.concat(dfs, ignore_index=True)


### 실제 처리된 데이터

In [2]:
df_final

,day,hour,강수,기온,뇌전,습도,풍속,풍향,하늘상태,month,year,읍면동
0,1,0.0,0.0,17.400000,-1.0,44.0,53.0,390.0,1.0,6.0,2010,중앙동
1,1,100.0,0.0,18.700001,-1.0,39.0,53.0,390.0,1.0,6.0,2010,중앙동
2,1,200.0,0.0,19.900000,-1.0,37.0,53.0,390.0,1.0,6.0,2010,중앙동
3,1,300.0,0.0,20.600000,-1.0,35.0,53.0,390.0,1.0,6.0,2010,중앙동
4,1,400.0,0.0,21.700001,-1.0,33.0,53.0,390.0,1.0,6.0,2010,중앙동
...,...,...,...,...,...,...,...,...,...,...,...,...
8251387,30,1900.0,0.0,5.600000,NaN,94.0,0.0,0.0,NaN,4.0,2022,목상동
8251388,30,2000.0,0.0,5.300000,NaN,95.0,0.0,230.0,NaN,4.0,2022,목상동
8251389,30,2100.0,0.0,8.200000,NaN,93.0,0.0,0.0,NaN,4.0,2022,목상동
8251390,30,2200.0,0.0,11.100000,NaN,66.0,1.8,200.0,NaN,4.0,2022,목상동


In [3]:
# 읍면동의 수가 142개로 맞게 나오는 것을 확인
len(list(df_final['읍면동'].unique()))

79

In [4]:
df_final.rename(columns = {'강수' : 'rainfall','기온' : 'temp','뇌전' : 'lightning', '습도':'humidity', '풍향':'wind_direction','풍속':'wind_speed','하늘상태':'condition','읍면동':'region'}, inplace = True)
df_final

,day,hour,rainfall,temp,lightning,humidity,wind_speed,wind_direction,condition,month,year,region
0,1,0.0,0.0,17.400000,-1.0,44.0,53.0,390.0,1.0,6.0,2010,중앙동
1,1,100.0,0.0,18.700001,-1.0,39.0,53.0,390.0,1.0,6.0,2010,중앙동
2,1,200.0,0.0,19.900000,-1.0,37.0,53.0,390.0,1.0,6.0,2010,중앙동
3,1,300.0,0.0,20.600000,-1.0,35.0,53.0,390.0,1.0,6.0,2010,중앙동
4,1,400.0,0.0,21.700001,-1.0,33.0,53.0,390.0,1.0,6.0,2010,중앙동
...,...,...,...,...,...,...,...,...,...,...,...,...
8251387,30,1900.0,0.0,5.600000,NaN,94.0,0.0,0.0,NaN,4.0,2022,목상동
8251388,30,2000.0,0.0,5.300000,NaN,95.0,0.0,230.0,NaN,4.0,2022,목상동
8251389,30,2100.0,0.0,8.200000,NaN,93.0,0.0,0.0,NaN,4.0,2022,목상동
8251390,30,2200.0,0.0,11.100000,NaN,66.0,1.8,200.0,NaN,4.0,2022,목상동


In [5]:
len(list(df_final['rainfall'].unique()))

349

In [6]:
df_final.to_csv("output/Daejeon.csv",mode='w',index=False,encoding='cp949')

In [7]:
del df_final